In [ ]:
import polars as pl
import numpy as np
import polars_ds as pld

In [ ]:
import polars_ds as pld
size = 100_000
df = pl.DataFrame({
    "id": range(size),
    "val1": np.random.random(size=size), 
    "val2": np.random.random(size=size), 
    "val3": np.random.random(size=size),
    "r": np.random.random(size=size),
    "rh": np.random.random(size=size)*10,
}).with_columns(
    pl.col("id").cast(pl.UInt64),
    pl.col("id").stats.rand_str().alias("s1"),
    pl.col("id").stats.rand_str().alias("s2"),
    pl.col("id").stats.rand_null(0.5)
)

In [ ]:
df.head(20)

In [ ]:
df.head()

In [ ]:
%%timeit
df.select(
    pl.col("s1").str2.similar_words(pl.col("s2"))
) # 718

In [ ]:

df.with_columns(
    pld.query_nb_cnt(
        0.1, # radius 
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        dist = "inf", # L Infinity distance 
        parallel = False 
    ).alias("nb_l_inf_cnt")
).head()